In [2]:
import matplotlib.pyplot as plt
import glob
import os
import mPyPl as mp
from pipe import *
from moviepy.editor import *
import numpy as np
import cv2

In [3]:
def close_clip(video):
    video.reader.close()
    video.audio.reader.close_proc()
    
    
def select_frames(video, target_num_frames):
    num_frames = video.shape[0]
    stride = num_frames // (target_num_frames - 1)
    return video[::stride]
    

def pad_video(video):
    height = video.shape[1]
    width = video.shape[2]
    target_size = width
    remainder = height % 2
    top = (target_size - height) // 2
    bottom = top + remainder
    left = 0
    right = 0
    
    if height > width:
        target_size = height        
        remainder = width % 2
        top = 0
        bottom = 0
        left = (target_size - width) // 2
        right = left + remainder
    
    target_shape = (video.shape[0], target_size, target_size, video.shape[3])
    
    output = np.zeros(target_shape, dtype=video.dtype)   
        
    for idx in range(video.shape[0]):
        output[idx] = cv2.copyMakeBorder(
            video[idx],
            top, bottom, left, right,
            cv2.BORDER_CONSTANT,
            value=0
        )
    return output

In [85]:
INPUTS_DIR = "X:\data" 
classes = { 'shots' : 1 }

data = (mp.get_datastream(os.path.join(INPUTS_DIR), classes=classes, ext=".resized.mp4")
 | mp.take(10)
 | mp.apply('filename','clip',lambda fn: VideoFileClip(fn).fx(vfx.resize, width=112))#newsize=(112,112)))
 | mp.apply('clip','video',lambda c: np.array(list(c.iter_frames())))
 | mp.apply('video','video', lambda v: pad_video(select_frames(v, 16)))
 | mp.iter('clip', close_clip)
 | mp.delfield('clip')
 | mp.as_list
)

In [86]:
shots = [x['video'] for x in data if x['class_id'] == 1]
shots_arr = np.asarray(shots, dtype=np.float32)
shots_arr.shape

(10, 16, 112, 112, 3)

In [87]:
from keras.models import load_model

In [88]:
model = load_model("X:\models\c3d_v3-unfreezed-big-chkpt-09-0.29-0.86.hdf5")

In [89]:
y_pred = model.predict(shots_arr)

In [90]:
y_pred

array([[2.0430427e-05],
       [7.3875946e-01],
       [5.9001303e-01],
       [6.6608936e-01],
       [6.7096251e-01],
       [5.6932247e-01],
       [6.8388402e-01],
       [7.7740335e-01],
       [5.9376562e-01],
       [8.0818975e-01]], dtype=float32)